# Writing an Algorithm that Uses DMAC Trading

## Data Loading

In [21]:
# Initial imports
import numpy as np
import os
import csv
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import warnings
from pathlib import Path
import datetime as dt

#Subsequent imports
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
%matplotlib inline

In [22]:
pip install alpaca-trade-api

In [23]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [24]:
# List the ticker symbols for the stocks of interest
lst=['BMO.TO', 'BNS.TO', 'CM.TO', 'NA.TO','RY.TO', 'TD.TO', 'ZEB.TO','ZWB.TO', 'XGB.TO', 'CL' ]

# Determine the start and end dates
start_date = '2016-08-01'
end_date = '2019-07-31'

# Create empty dataframe to hold stock data
dfAllStocks_Yahoo=pd.DataFrame()

# Gathers stock data from listed tickers and combines them into a dataframe
for ticker in lst:
    # Create data frame from API in loop 
    df= yf.download(ticker, start= start_date, end= end_date)
    # Remove to '.TO' suffix from Canadian stocks
    tic= ticker.replace('.TO','')
    df['Ticker']=tic
    # Drop unwanted coloumns - Keep Adjusted Close
    df.drop(columns= ["Open", "High", "Low", "Volume", "Close", "Ticker"], inplace= True)
    # Combine individual stocks into a single data frame
    dfAllStocks_Yahoo=pd.concat([dfAllStocks_Yahoo, df], axis= 1)
    
  # Rename the coloumns to the ticker symbols in the list
dfAllStocks_Yahoo.columns = lst
dfAllStocks_Yahoo.index.name = 'Date'

# Drop the N/As
dfAllStocks_Yahoo = dfAllStocks_Yahoo.dropna()

# Drop the time component of the date
dfAllStocks_Yahoo.index = dfAllStocks_Yahoo.index.date

# Drop the time component of the date
#dfAllStocks_Yahoo.index = dfAllStocks_Yahoo.index.date

# Display the data
display(dfAllStocks_Yahoo)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,BMO.TO,BNS.TO,CM.TO,NA.TO,RY.TO,TD.TO,ZEB.TO,ZWB.TO,XGB.TO,CL
2016-08-02,64.213966,48.621632,35.759655,34.783409,61.751713,43.958851,18.372488,11.604877,19.594658,64.647606
2016-08-03,64.019852,48.584633,35.924675,34.838516,61.869514,43.943207,18.388443,11.611920,19.568819,63.965481
2016-08-04,63.755814,48.503189,35.994354,34.751907,61.877350,43.896263,18.348564,11.597816,19.637751,64.569878
2016-08-05,64.338234,49.125111,36.434425,35.058968,62.395618,44.498657,18.539942,11.703639,19.577442,64.397194
2016-08-08,64.835236,49.362034,36.599449,35.673065,62.717560,44.623833,18.691444,11.788288,19.594658,64.267662
...,...,...,...,...,...,...,...,...,...,...
2019-07-24,85.933426,59.187160,43.932133,57.061749,92.304749,67.621864,25.332594,15.127074,20.644247,66.042404
2019-07-25,85.967873,59.557652,43.902519,56.945850,91.830200,67.360191,25.297422,15.110668,20.647020,66.644653
2019-07-26,86.097046,59.397671,43.817932,57.195465,92.067474,67.368935,25.315014,15.127074,20.637760,68.275314
2019-07-29,86.320930,59.616600,43.779877,57.088490,92.067474,67.499756,25.332655,15.139423,20.591444,70.017181


In [ ]:
# Read the aapl.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
#aapl_df = pd.read_csv(
    #Path("../Resources/aapl.csv"),
    #index_col="date",
    #parse_dates=True,
    #infer_datetime_format=True
)

In [ ]:
# Review the DataFrame
#aapl_df.head()

In [25]:
# Filter the date index and close columns
signals_df = dfAllStocks_Yahoo.loc[:,["BNS.TO"]]

# Review the DataFrame
signals_df.head()

,BNS.TO
2016-08-02,48.621632
2016-08-03,48.584633
2016-08-04,48.503189
2016-08-05,49.125111
2016-08-08,49.362034


In [26]:
# Use hvplot to visualize the data
signals_df.hvplot()

:Curve   [index]   (BNS.TO)

## SMA for the Short and Long Windows Calculation

In [27]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

In [28]:
# Generate the short and long window simple moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["BNS.TO"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["BNS.TO"].rolling(window=long_window).mean()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,BNS.TO,SMA50,SMA100
2016-08-02,48.621632,NaN,NaN
2016-08-03,48.584633,NaN,NaN
2016-08-04,48.503189,NaN,NaN
2016-08-05,49.125111,NaN,NaN
2016-08-08,49.362034,NaN,NaN


,BNS.TO,SMA50,SMA100
2019-07-24,59.187160,58.913610,59.378772
2019-07-25,59.557652,58.901594,59.373466
2019-07-26,59.397671,58.906174,59.370423
2019-07-29,59.616600,58.911307,59.368829
2019-07-30,59.347137,58.915542,59.367252


## Trading Signals Identification

In [29]:
# Create a column to hold the trading signal
signals_df["Signal"] = 0.0

In [30]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
signals_df.tail(10)

,BNS.TO,SMA50,SMA100,Signal
2019-07-17,58.159859,59.109381,59.504423,0.0
2019-07-18,58.412479,59.056497,59.468277,0.0
2019-07-19,58.412479,59.002781,59.433118,0.0
2019-07-22,58.404057,58.957714,59.397545,0.0
2019-07-23,58.791386,58.928544,59.384331,0.0
2019-07-24,59.187160,58.913610,59.378772,0.0
2019-07-25,59.557652,58.901594,59.373466,0.0
2019-07-26,59.397671,58.906174,59.370423,0.0
2019-07-29,59.616600,58.911307,59.368829,0.0
2019-07-30,59.347137,58.915542,59.367252,0.0


## Finding the Crossover Points

In [31]:
# Slice the DataFrame to confirm the Signal
signals_df.loc["2016-08-02":"2019-07-30"]

TypeError: '<' not supported between instances of 'datetime.date' and 'str'

In [ ]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.loc["2015-02-09":"2015-02-17"]

In [ ]:
# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="yellow",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
exit

In [ ]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
entry

In [ ]:
# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
security_close

In [ ]:
# Visualize moving averages
moving_avgs = signals_df[["SMA50", "SMA100"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
moving_avgs

In [ ]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)